In [ ]:
import wandb
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np


In [2]:
wandb.login(key="8a88a8c49d1c2d31b8677fe0b8eb7d3e3a031f83")
api = wandb.Api()
fill_value = "N/A"

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /export/home/0schindl/.netrc
wandb: Currently logged in as: belaschindler (belaschindler-university-hamburg) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [3]:
sweep_id = "belaschindler-university-hamburg/0schindl-LayUp_sweeps_question1_selection_method/wk4w5q0t"
sweep2_id = "belaschindler-university-hamburg/0schindl-LayUp_sweeps_question1_selection_method/6uix7luf"
sweep3_id = "belaschindler-university-hamburg/0schindl-LayUp_sweeps_question1_selection_method/lpeje9a7"
sweep = api.sweep(sweep_id)
sweep2 = api.sweep(sweep2_id)
sweep3 = api.sweep(sweep3_id)
runs1 = sweep.runs
runs2 = sweep2.runs
runs3 = sweep3.runs
cil_datasets1 = ["cifar100", "cub", "imageneta", "vtab", "omnibenchmark"]
cil_datasets2 =  ["imagenetr", "cars"]
dil_datasets = ["cddb", "dil_imagenetr", "limited_domainnet"]


In [19]:
def get_forgetting(runs, dataset_names, data):
    for run in runs:
        config = run.config
        summary = run.summary
        
        dataset = config.get("dataset")
        selection_method = config.get("selection_method")
        run_id = run.id
        state = run.state
        num_E = config.get("moe_max_experts")



        if state == "finished" and dataset in dataset_names and (selection_method == "around" or selection_method == "inv_ws_div"):
            # Get the history of the run
            history = run.history()

            run_data = {
                "run_id": run_id,
                "dataset": dataset,
                "selection_method": selection_method           
                }

            # End accuracy of tasks
            for key, value in summary.items():
                if key.startswith("task_") and key.endswith("/acc") and key != "task_mean/acc" and key != "task_wmean/acc":
                    t = int(key.split("_")[1].split("/")[0])

                    # is an initial task
                    if num_E <= t:
                        continue

                    key = "end_" + key
                    run_data[key] = value

            # Initial accuracy of tasks
            for line in history.items():
                key = line[0]
                if key.startswith("task_") and key.endswith("/acc") and key != "task_mean/acc" and key != "task_wmean/acc":
                    t = int(key.split("_")[1].split("/")[0])

                    # is an initial task
                    if num_E <= t:
                        continue

                    values = line[1]
                    filtered_values = values[~np.isnan(values)]
                    index = num_E - t - 1
                    base_acc = filtered_values.iloc[index]
                    key = "initial_" + key
                    run_data[key] = base_acc
            

            data.append(run_data)
    return data

def get_bwt(runs, dataset_names, data):
    for run in runs:
        config = run.config
        summary = run.summary
        
        dataset = config.get("dataset")
        selection_method = config.get("selection_method")
        run_id = run.id
        state = run.state
        num_E = config.get("moe_max_experts")



        if state == "finished" and dataset in dataset_names:
            # Get the history of the run
            history = run.history()

            run_data = {
                "run_id": run_id,
                "dataset": dataset,
                "selection_method": selection_method           
                }

            # End accuracy of tasks
            for key, value in summary.items():
                if key.startswith("task_") and key.endswith("/acc") and key != "task_mean/acc" and key != "task_wmean/acc":
                    t = int(key.split("_")[1].split("/")[0])

                    # is an initial task
                    if num_E <= t:
                        continue

                    key = "end_" + key
                    run_data[key] = value

            # Initial accuracy of tasks
            for line in history.items():
                key = line[0]
                if key.startswith("task_") and key.endswith("/acc") and key != "task_mean/acc" and key != "task_wmean/acc":
                    t = int(key.split("_")[1].split("/")[0])

                    # is an initial task
                    if num_E <= t:
                        continue

                    values = line[1]
                    filtered_values = values[~np.isnan(values)]
                    index = num_E - t - 1
                    base_acc = filtered_values.iloc[index]
                    key = "initial_" + key
                    run_data[key] = base_acc
            

            data.append(run_data)
    data = pd.DataFrame(data)
    return data

def get_average_forgetting(df, num_tasks=5):
    forgetting_values = []
    for index, row in df.iterrows():
        row_forgetting = []
        for i in range(num_tasks):  # Assuming there are tasks 0 to 4
            end_col = f'end_task_{i}/acc'
            initial_col = f'initial_task_{i}/acc'
            if end_col in row and initial_col in row and pd.notna(row[end_col]) and pd.notna(row[initial_col]):
                forgetting = row[initial_col] - row[end_col]
                row_forgetting.append(forgetting)
        # Calculate the average forgetting for the row, ignoring NaNs
        avg_forgetting = np.nanmean(row_forgetting)
        forgetting_values.append(avg_forgetting)
    return forgetting_values

In [21]:
print("####### 1 #######")
data = get_forgetting(runs1, cil_datasets1, [])
print("####### 2 #######")
data = get_forgetting(runs2, cil_datasets2, data)
print("####### 3 #######")
data = get_forgetting(runs3, dil_datasets, data)

# Add the average forgetting as a new column
data = pd.DataFrame(data)
data['average_forgetting'] = get_average_forgetting(data)

print("\nDataFrame der Sweep-Runs:")
pd.set_option('display.width', 5000)
pd.set_option('display.max_colwidth', None)
print(data.sort_values(by=['dataset', "selection_method"]))
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


####### 1 #######
####### 2 #######
####### 3 #######

DataFrame der Sweep-Runs:
      run_id            dataset selection_method  end_task_0/acc  end_task_1/acc  end_task_2/acc  end_task_3/acc  end_task_4/acc  initial_task_3/acc  initial_task_2/acc  initial_task_0/acc  initial_task_1/acc  initial_task_4/acc  average_forgetting
10  doicok2o               cars           around        0.487256        0.295285        0.260024        0.252370        0.409478            0.428910            0.410693            0.623688            0.387097            0.550425            0.139280
11  k4z61mie               cars       inv_ws_div        0.533733        0.325062        0.323208        0.323460        0.357230            0.464455            0.448360            0.619190            0.359801            0.411908            0.088204
13  5jdhgw9k               cddb           around        0.599000        0.827500        0.597673        0.582500        0.477778            0.582500            0.597673    

In [22]:
print("####### BWT FROM DIL #######")
data = get_bwt(runs3, dil_datasets, [])
data["average_forgetting"] = get_average_forgetting(data)
pd.set_option('display.width', 5000)
pd.set_option('display.max_colwidth', None)
print(data)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

####### BWT FROM DIL #######
      run_id            dataset selection_method  end_task_0/acc  end_task_1/acc  end_task_2/acc  end_task_3/acc  end_task_4/acc  initial_task_0/acc  initial_task_1/acc  initial_task_3/acc  initial_task_4/acc  initial_task_2/acc  average_forgetting
0   k3hs9nv8               cddb       inv_ws_div        0.599000        0.827500        0.597673        0.582500        0.477778            0.599000            0.827500            0.582500            0.477778            0.597673            0.000000
1   1v2sm0we               cddb           ws_div        0.599000        0.827500        0.597673        0.582500        0.477778            0.599000            0.827500            0.582500            0.477778            0.597673            0.000000
2   k9tor776               cddb       inv_kl_div        0.599000        0.827500        0.597673        0.582500        0.477778            0.599000            0.827500            0.582500            0.477778            0.59